In [2]:
import pandas as pd
from transformers import pipeline

In [4]:
# Wczytaj dane
df = pd.read_csv('dara_reviews.csv')  # Zastąp 'opinions.csv' ścieżką do swojego pliku
texts = df['text_EN'].dropna().tolist()


In [7]:
# Inicjalizacja modelu do analizy sentymentu
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# Analiza sentymentu
results = sentiment_analyzer(texts)

In [9]:
# Dodanie wyników do DataFrame
df['sentiment'] = [result['label'] for result in results]
df['score'] = [result['score'] for result in results]


In [11]:
df[df.stars == 1]

,Unnamed: 0,title,city,address,stars,text,textTranslated,text_PL,text_EN,sentiment,score
8,24,Dara kebab,Rzeszów,"Leska 2, 35-505 Rzeszów, Polska",1,"Nie polecam,w kebabie miałem włosa..",NaN,"Nie polecam,w kebabie miałem włosa..",I don't recommend that I had hair in kebab.,LABEL_1,0.506099
10,26,Dara kebab,Rzeszów,"Leska 2, 35-505 Rzeszów, Polska",1,"Nie polecam, może i mięso się trochę polepszył...",NaN,"Nie polecam, może i mięso się trochę polepszył...","I don't recommend it, maybe the meat got a lit...",LABEL_1,0.504905
21,45,Dara Kebab,Rzeszów,"Obrońców Poczty Gdańskiej 14, 35-509 Rzeszów, ...",1,Jedzenie super: placisz jedynie za mieso a dos...,NaN,Jedzenie super: placisz jedynie za mieso a dos...,Super food: you only pay for meat and you get:...,LABEL_1,0.533368
24,48,Dara Kebab,Rzeszów,"Obrońców Poczty Gdańskiej 14, 35-509 Rzeszów, ...",1,"Kiedyś pyszny, dziś w zamowieniu przyszedł cał...",NaN,"Kiedyś pyszny, dziś w zamowieniu przyszedł cał...","Once delicious, today in order came all soaked...",LABEL_1,0.519887
25,50,Dara Kebab,Rzeszów,"Obrońców Poczty Gdańskiej 14, 35-509 Rzeszów, ...",1,Niedobry kebab wręcz paskudny cały przemoknięt...,NaN,Niedobry kebab wręcz paskudny cały przemoknięt...,Bad kebab even nasty whole soaked with sauces ...,LABEL_1,0.515514
...,...,...,...,...,...,...,...,...,...,...,...
4804,9895,Dara Kebab,Rzeszów,"Rynek 18, 35-064 Rzeszów, Polska",1,"w kebabie brakowalo sosow, a sam w sobie w sma...",NaN,"w kebabie brakowalo sosow, a sam w sobie w sma...","in kebab there were no sauces, and in itself t...",LABEL_1,0.514400
4811,9909,Dara kebab,Rzeszów,"Aleja Majora Wacława Kopisto 1, 35-515 Rzeszów...",1,Kebab dobry lecz słabo upieczone ciasto i rozw...,NaN,Kebab dobry lecz słabo upieczone ciasto i rozw...,Kebab good but poorly baked cake and smashed,LABEL_1,0.513576
4825,9938,Dara Kebab,Rzeszów,"Grunwaldzka 24, 35-068 Rzeszów, Polska",1,Wróciłem po latach. Jest wiele lepszych miejsc...,NaN,Wróciłem po latach. Jest wiele lepszych miejsc...,"I've come back years later, and there's a lot ...",LABEL_1,0.515086
4830,9946,Dara Kebab,Rzeszów,"Rynek 18, 35-064 Rzeszów, Polska",1,Kiedyś najlepszy w Polsce\nTeraz średni\nDo te...,NaN,Kiedyś najlepszy w Polsce\nTeraz średni\nDo te...,Once the best in Poland Now average For this p...,LABEL_1,0.509375


In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
# Załaduj dane
data = pd.read_csv("dara_reviews.csv")  # Zamień na właściwą ścieżkę do pliku

In [20]:
# Zakładamy, że kolumna 'text_EN' zawiera tekst, a 'label' to 0/1 (negatywny/pozytywny)
texts = data['text_EN'].astype(str)
data['label'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)
labels = data['label']

# Tokenizacja i przekształcenie tekstu na sekwencje
max_words = 10000  # Maksymalna liczba unikalnych słów
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding sekwencji
max_length = 100  # Maksymalna długość tekstu
X = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Budowa modelu
embedding_dim = 64
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Kompilacja modelu
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [26]:
# Trenowanie modelu
epochs = 10
batch_size = 32
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.8009 - loss: 0.4645 - val_accuracy: 0.9148 - val_loss: 0.2266
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.9190 - loss: 0.2209 - val_accuracy: 0.9187 - val_loss: 0.1973
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.9560 - loss: 0.1235 - val_accuracy: 0.9187 - val_loss: 0.2070
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.9740 - loss: 0.0792 - val_accuracy: 0.9368 - val_loss: 0.1885
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.9804 - loss: 0.0612 - val_accuracy: 0.9355 - val_loss: 0.2100
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.9858 - loss: 0.0358 - val_accuracy: 0.9290 - val_loss: 0.2612
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.9913 - loss: 0.0289 - val_accuracy: 0.9316 - val_loss: 0.2567
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.9975 - loss: 0.0161 - val_accuracy: 0.9290 - 

In [27]:
# Ewaluacja modelu
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {accuracy:.2f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9240 - loss: 0.3027
Test Accuracy: 0.93


In [28]:
# Predykcja i raport
predictions = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, predictions))

31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       194
           1       0.95      0.97      0.96       775

    accuracy                           0.93       969
   macro avg       0.90      0.88      0.89       969
weighted avg       0.93      0.93      0.93       969



In [30]:
model_opinion = (model.predict(X) > 0.5).astype(int)

152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step


In [33]:
data['model_opinion'] = model_opinion

In [54]:
df_opinions = pd.DataFrame(data.groupby('address').sum('model_opinion')['model_opinion'] / data.groupby('address').count()['model_opinion'])
df_opinions['rank_opinion'] = df_opinions['model_opinion'].rank(method='dense', ascending=False).astype(int)
df_opinions

,model_opinion,rank_opinion
address,,
"Aleja Majora Wacława Kopisto 1, 35-515 Rzeszów, Polska",0.968142,1
"Generała Mariana Langiewicza 69, 35-021 Rzeszów, Polska",0.816901,7
"Grunwaldzka 24, 35-068 Rzeszów, Polska",0.731626,11
"Hetmańska 42, 35-078 Rzeszów, Polska",0.894531,4
"Ignacego Łukasiewicza 78, 35-604 Rzeszów, Polska",0.896175,3
"Jana Matejki 16, 35-060 Rzeszów, Polska",0.740586,10
"Kwiatkowskiego, 35-311 Rzeszów, Polska",0.664671,13
"Leska 2, 35-505 Rzeszów, Polska",0.809278,8
"Obrońców Poczty Gdańskiej 14, 35-509 Rzeszów, Polska",0.710843,12


In [56]:
df_stars = pd.DataFrame(data.groupby('address').mean('stars')['stars'])
df_stars['rank_stars'] = df_stars['stars'].rank(method='dense', ascending=False).astype(int)
df_stars

,stars,rank_stars
address,,
"Aleja Majora Wacława Kopisto 1, 35-515 Rzeszów, Polska",4.883186,1
"Generała Mariana Langiewicza 69, 35-021 Rzeszów, Polska",4.239437,8
"Grunwaldzka 24, 35-068 Rzeszów, Polska",4.027840,9
"Hetmańska 42, 35-078 Rzeszów, Polska",4.605469,3
"Ignacego Łukasiewicza 78, 35-604 Rzeszów, Polska",4.573770,4
"Jana Matejki 16, 35-060 Rzeszów, Polska",3.993026,11
"Kwiatkowskiego, 35-311 Rzeszów, Polska",3.688623,13
"Leska 2, 35-505 Rzeszów, Polska",4.311856,6
"Obrońców Poczty Gdańskiej 14, 35-509 Rzeszów, Polska",3.960843,12


In [65]:
merged_df = pd.merge(df_opinions, df_stars, left_index=True, right_index=True)
merged_df['opinions_count'] = data.groupby('address').count()['title']
merged_df['stars_normalized'] = merged_df['stars'] / 5
merged_df.sort_values(by='rank_opinion')

,model_opinion,rank_opinion,stars,rank_stars,opinions_count,stars_normalized
address,,,,,,
"Aleja Majora Wacława Kopisto 1, 35-515 Rzeszów, Polska",0.968142,1,4.883186,1,565,0.976637
"Podkarpacka 12A, 35-082 Rzeszów, Polska",0.925714,2,4.725714,2,175,0.945143
"Ignacego Łukasiewicza 78, 35-604 Rzeszów, Polska",0.896175,3,4.573770,4,183,0.914754
"Hetmańska 42, 35-078 Rzeszów, Polska",0.894531,4,4.605469,3,256,0.921094
"Rynek 18, 35-064 Rzeszów, Polska",0.840937,5,4.368681,5,811,0.873736
"Witolda 8b, 35-302 Rzeszów, Polska",0.820312,6,4.281250,7,128,0.856250
"Generała Mariana Langiewicza 69, 35-021 Rzeszów, Polska",0.816901,7,4.239437,8,71,0.847887
"Leska 2, 35-505 Rzeszów, Polska",0.809278,8,4.311856,6,388,0.862371
"al. Tadeusza Rejtana 69, 35-001 Rzeszów, Polska",0.745098,9,4.019608,10,153,0.803922


In [66]:
# Zapisanie modelu
model.save("sentiment_model.h5")

# Zapisanie tokenizatora
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Model i tokenizator zapisane.")

Model i tokenizator zapisane.
